<a href="https://colab.research.google.com/github/atharahmad000/atharahmad000/blob/main/Semantic_Search_Using_Open_Source_Transformers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
dataset = load_dataset("xtreme", "MLQA.en.en")

# rename test -> train and val -> test (as we will use it in later in this chapter)
dataset['train'] = dataset['test']
dataset['test'] = dataset['validation']
del dataset['validation']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/7.49M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/724k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/11590 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1148 [00:00<?, ? examples/s]

In [4]:
bi_encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
docs= dataset['test']['context']
doc_emb = bi_encoder.encode(docs, batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

In [6]:
doc_emb.shape

(1148, 768)

In [7]:
from sentence_transformers.util import semantic_search

In [8]:
# function to find most similar documents
def find_similar(text, embeddings, documents, k=5):
    query_emb = bi_encoder.encode([text], show_progress_bar=False)
    similarities = semantic_search(query_emb, embeddings, top_k=k)
    return [(documents[sim['corpus_id']],sim['score'], sim['corpus_id'] )for sim in similarities[0]]


In [9]:
from random import sample

In [10]:
query= sample(dataset['test']['question'],1)[0]

In [11]:
print(query)

What is the name of the neurotransmitter that he is known for?


In [12]:
import hashlib
def my_hash(s):
  return hashlib.md5(s.encode()).hexdigest()

In [13]:
q_to_hash= {data['question']: my_hash(data['context']) for data in dataset['test']}

In [14]:
from sentence_transformers import SentenceTransformer, util

In [15]:
from sentence_transformers import CrossEncoder

In [16]:
import torch.nn as nn
import numpy as np

In [19]:
def eval_ranking_open(query, top_k=3, re_rank_model= None):
  ans={'retrieved_correct_position': None}
  correct_hash= q_to_hash[query]
  results= find_similar(query, doc_emb, docs, k=top_k)
  for idx, (passage, score, passage_id) in enumerate(results):
    if correct_hash== my_hash(passage):
      ans['retrieved_correct_position']= idx
    if re_rank_model is not None:
      ans['reranked_correct_position']= None
      sentence_combinations= [(query, r[0]) for r in results ]
      similarity_scores= re_rank_model.predict(sentence_combinations, activation_fct= nn.Sigmoid())
      sim_scores_argsort= list(reversed(np.argsort(similarity_scores)))
      for i, idx in enumerate(sim_scores_argsort):
        r= results[idx]
        if correct_hash and correct_hash== my_hash(r[0]):
          ans['reranked_correct_position']= i

  return ans

In [20]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', num_labels=1)

eval_ranking_open(query, top_k=3, re_rank_model=cross_encoder)

{'retrieved_correct_position': 0, 'reranked_correct_position': 0}